In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from tqdm import tqdm, tqdm_notebook
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import gensim
from gensim import corpora, models
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from prettytable import PrettyTable
import scipy.sparse
import scipy
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
print('CWD is ', os.getcwd())

# Vis Libs..

# Text Libs..

# ML Libs...

CWD is  d:\NEU\Năm 3\SEMESTER 6\ĐỀ ÁN DSEB\Mercari Price Prediction\3. Modelling


# 6. Data Merge

**Getting train data**

In [70]:
file = open('../2. Processing/storage/train_bow_name', 'rb')
train_bow_name = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_bow_desc', 'rb')
train_bow_desc = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_brand', 'rb')
train_brand = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_category', 'rb')
train_category = pickle.load(file)
file.close()

file = open('../2. Processing/storage/train_dummies', 'rb')
train_dummies = pickle.load(file)
file.close()

# file = open('../2. Processing/storage/y_train', 'rb')
# y_train = pickle.load(file)
# file.close()

In [71]:
file = open('../2. Processing/storage/df_train_model', 'rb')
df_train_model = pickle.load(file)
file.close()

y_train = np.log1p(df_train_model['price'])

**Getting test data**

In [72]:
file = open('../2. Processing/storage/test_bow_name', 'rb')
test_bow_name = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_bow_desc', 'rb')
test_bow_desc = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_brand', 'rb')
test_brand = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_category', 'rb')
test_category = pickle.load(file)
file.close()

file = open('../2. Processing/storage/test_dummies', 'rb')
test_dummies = pickle.load(file)
file.close()

In [73]:
file = open('../2. Processing/storage/df_test_model', 'rb')
df_test_model = pickle.load(file)
file.close()


y_test = np.log1p(df_test_model['price'])

In [74]:
df_test_model

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
690830,690830,Armani exchange v neck bundle,3,Men/Tops/T-shirts,A|X Armani Exchange,26.0,0,"Armani exchange v neck tee , black and gold le..."
41112,41112,FREE SHIP Kylie Jenner Demi Wispies,1,Beauty/Makeup/Eyes,NaN,6.0,1,FAST AND FREE SHIPPING High quality 5 pairs
1397662,1397662,Reserved,2,Women/Sweaters/Hooded,Calvin Klein,34.0,0,Size small New without tags Cropped hoodie & s...
1100352,1100352,All in One Milk,1,Beauty/Hair Care/Conditioners,NaN,10.0,1,Davines all in one milk. Hair treatment. Trial...
1296737,1296737,*reserved* bath and body works set,1,Beauty/Bath & Body/Bath,Bath & Body Works,37.0,1,No description yet
...,...,...,...,...,...,...,...,...
918564,918564,Jordan Eclipse Girls Tennis Shoes,2,Women/Shoes/Athletic,Nike,40.0,1,Super cute peach and blue Jordan Eclipse tenni...
1196120,1196120,Stash Sup perfume,2,Beauty/Fragrance/Women,NaN,22.0,1,Stash Sup perfume Strong scent. Full in box. [...
924186,924186,Emerica Baja Shoes,3,Women/Shoes/Fashion Sneakers,NaN,14.0,1,Most comfortable sneakers ever.
1232862,1232862,Pink sweatshirt,2,Women/Sweaters/Hooded,PINK,29.0,0,Leopard print


In [75]:
print("train_bow_name shape:", train_bow_name.shape)
print("train_bow_desc shape:", train_bow_desc.shape)
print("train_brand shape:", train_brand.shape)
print("train_category shape:", train_category.shape)
print("train_dummies shape:", train_dummies.shape)

train_bow_name shape: (1111901, 31861)
train_bow_desc shape: (1111901, 100000)
train_brand shape: (1111901, 4463)
train_category shape: (1111901, 940)
train_dummies shape: (1111901, 7)


**Concatenate all the features that we generated using scipy.sparse.hstack function and converts the datatype in to 'float32' format so as to feed in ML models**

In [76]:
X_train = scipy.sparse.hstack((train_bow_name, train_bow_desc,
                               train_brand, train_category, train_dummies
                               )).tocsr().astype('float32')

X_test = scipy.sparse.hstack((test_bow_name, test_bow_desc,
                              test_brand, test_category, test_dummies
                              )).tocsr().astype('float32')

print('='*50)
print("X_train Shape is ", X_train.shape)
print("y_train Shape is ", y_train.shape)

print('='*50)
print("X_test Shape is ", X_test.shape)
print("y_test Shape is ", y_test.shape)

X_train Shape is  (1111901, 137271)
y_train Shape is  (1111901,)
X_test Shape is  (370634, 137271)
y_test Shape is  (370634,)


In [77]:
file = open('final_processing/X_train', 'wb')
pickle.dump(X_train, file)
file.close()

file = open('final_processing/y_train', 'wb')
pickle.dump(y_train, file)
file.close()

file = open('final_processing/X_test', 'wb')
pickle.dump(X_test, file)
file.close()

file = open('final_processing/y_test', 'wb')
pickle.dump(y_test, file)
file.close()

# 7. Evaluation Metrics

**Root Mean Squared Logarithm Error (RMLSE):**

$$\epsilon = \sqrt{\frac{1}{n}\sum_{i=1}^n(log(p_i+1) - log(a_i+1)^2)}$$

Where: 
- $\epsilon$ is the RMSLE value (score)
- $\mathbf{n}$ is the total number of observations in the (public/private) data set
- $p_i$ is your prediction of price
- $a_i$ is the acutual sale price for $i$
- $log(x)$ is the natural logarithm of $x$

**This code creates the RMSLE scorer function which would be used as Evalation metric for ML models**

In [78]:
def rmsle_score(y, y_pred):
    y = y.reset_index(drop=True)
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1))** 2.0
               for i in range(len(y_pred))]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [79]:
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle_score(preds, labels), False

In [80]:
# Function for RMSE - (Root mean Square error)
def rmse_score(y, y_pred):
    return math.sqrt(mean_squared_error(y, y_pred))

# 8. Baseline Ridge Model

## 8.1. Using Existing Hyperparameters

In [81]:
ridge_model = Ridge(solver='lsqr', fit_intercept=False)
ridge_model.fit(X_train, y_train)

Ridge(fit_intercept=False, solver='lsqr')

In [82]:
preds = np.expm1(ridge_model.predict(X_test))
print('Train RMSLE = ', rmsle_score(np.expm1(y_train), np.expm1(ridge_model.predict(X_train))))
print('Test RMSLE = ', rmsle_score(np.expm1(y_test), preds))
print('='*30)
print('Train RMSE = ', rmse_score(np.expm1(y_train),np.expm1(ridge_model.predict(X_train))))
print('Test RMSE = ', rmse_score(np.expm1(y_test) , preds))

Train RMSLE =  0.42896677837705344
Test RMSLE =  0.47763429814443636
Train RMSE =  28.104765259766562
Test RMSE =  35.09986317908642


In [83]:
list(preds)

[45.652798,
 11.501387,
 29.827457,
 12.477216,
 17.645397,
 11.679233,
 43.093006,
 21.24593,
 5.7625494,
 7.695904,
 71.360054,
 14.937467,
 9.920593,
 25.399088,
 14.004762,
 20.826714,
 50.842064,
 13.7785,
 17.658262,
 13.560741,
 29.916033,
 33.734016,
 6.9683833,
 11.338355,
 13.105425,
 14.773853,
 40.728218,
 27.130127,
 13.741072,
 11.300741,
 16.827017,
 12.50662,
 26.416958,
 19.068405,
 18.49569,
 9.462171,
 45.34379,
 23.706911,
 16.758825,
 16.742138,
 45.969814,
 42.696205,
 9.570555,
 13.180403,
 13.80345,
 25.9094,
 27.398277,
 13.26764,
 8.246903,
 11.586781,
 73.40943,
 32.384724,
 11.1192875,
 11.22582,
 22.006758,
 13.94931,
 19.401566,
 17.154295,
 14.271122,
 9.669882,
 20.831877,
 10.201734,
 7.695101,
 42.367954,
 19.15221,
 48.629414,
 16.05968,
 13.204113,
 20.868082,
 9.818891,
 24.04668,
 8.295579,
 14.433965,
 133.9883,
 12.769753,
 24.418833,
 21.625383,
 17.856321,
 3.636663,
 50.74277,
 11.729947,
 31.222218,
 19.568264,
 9.171647,
 11.319029,
 13.3307

In [84]:
y_test_exp = np.expm1(y_test)
y_test_exp.to_list()

[26.0,
 5.999999999999999,
 33.99999999999999,
 10.000000000000002,
 36.99999999999999,
 3.0,
 75.0,
 14.999999999999998,
 3.0,
 6.999999999999998,
 39.0,
 12.999999999999996,
 16.999999999999996,
 16.0,
 10.000000000000002,
 23.999999999999996,
 48.99999999999999,
 12.999999999999996,
 14.0,
 20.0,
 45.0,
 45.99999999999999,
 3.0,
 14.0,
 17.999999999999996,
 12.999999999999996,
 89.0,
 30.0,
 12.999999999999996,
 16.0,
 9.000000000000002,
 10.000000000000002,
 10.000000000000002,
 21.000000000000004,
 20.0,
 6.999999999999998,
 35.99999999999999,
 30.0,
 12.0,
 11.0,
 35.0,
 70.0,
 6.999999999999998,
 12.999999999999996,
 9.000000000000002,
 14.999999999999998,
 42.0,
 20.0,
 10.000000000000002,
 12.999999999999996,
 64.99999999999997,
 56.00000000000001,
 14.0,
 14.0,
 14.999999999999998,
 14.0,
 16.0,
 11.0,
 12.0,
 6.999999999999998,
 21.000000000000004,
 9.000000000000002,
 12.0,
 130.99999999999997,
 22.0,
 54.000000000000014,
 16.0,
 14.0,
 29.000000000000004,
 9.00000000000000

In [85]:
pred_result = pd.DataFrame(data = [y_test_exp.to_list(), list(preds)])
pred_result = pred_result.T
pred_result.columns = ['test_true','test_pred']
pred_result

,test_true,test_pred
0,26.0,45.652798
1,6.0,11.501387
2,34.0,29.827457
3,10.0,12.477216
4,37.0,17.645397
...,...,...
370629,40.0,31.854565
370630,22.0,16.706919
370631,14.0,16.187096
370632,29.0,20.583605


****
